In [2]:
#Imports
import requests as rq
import json
import requests
import urllib2
from bs4 import BeautifulSoup
import re
import string
from afinn import Afinn
import os
import matplotlib.pyplot as plt


In [3]:
#rapgenius information
rap_genius_id = "_17NJOxR9_RGp1XTQ1Vl3O--bxYh-cSzcCwASKlr-Xlyj3vh2_BsdedBZLX6xgdt"
client_secret = "2wlydcjFiqOlSI5Zi65OeizvaWRx16xkkjIBfPkqZ9t1aFNNYTrjza8ny6xYlXdJQSP_vMzj6e20O7ORtjIIcQ"
client_access_token = "Uzv_tj6uOXE9_1eX2wCrOGi3j4JtmYcm1kVF-UmPcC5gJh1MQZKJLph4aFp9frFx"

In [4]:
#pass the URL of rapgenius and get returned the song lyrics
def getLyrics (URL):
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

    # Scrape and return the song lyrics from the HTML
    return html.find("div", class_="lyrics").get_text()

***GETTING NEGATIVE WORDS***

dictionary of words is score_word_dict

In [5]:
#Get requests response "Failed to establish a new connection: [Errno -2]"
url_web = "https://raw.githubusercontent.com/hyharryhuang/AFINN-json/master/AFINN.json"
#We are going to hardcode the Afinn-111 json

afinn_data = rq.get(url_web).json()

In [6]:
score_word_dict = dict(afinn_data) #negative word dictionary

In [7]:
#add firearms
firearm_url = "https://genius.com/Rap-genius-firearms-and-firearm-companies-mentioned-in-rap-lyrics"
scrape_words = getLyrics(firearm_url)
firearm_list = scrape_words[130:].split("\n")

#add firearms in dictionary with sentiment score of -4
for firearm in firearm_list:
    score_word_dict[firearm.lower()] = -4

In [8]:
#add bulleted lists on rapgenius to a list that is returned and adds to score_word_dict
#needs score_word_dict to word, and adds the sentiment of that word (number)
def add_bulleted_lists(URL, number):
    words_drug = getLyrics(URL)
    first_list = words_drug.split("\n")
    new_list = []

    for x in first_list:
        new_list.append(x.replace(u'\u2022 ', ''))

    #add drugs in dictionary with sentiment score of -1
    for thing in new_list:
        score_word_dict[thing.lower()] = number

In [9]:
slang_drugs_url = "https://genius.com/Rap-genius-slang-dictionary-drugs-lyrics"
slang_firearms_url = "https://genius.com/Rap-genius-slang-dictionary-firearms-lyrics"

#adds words into sentiment dictionary
add_bulleted_lists(slang_drugs_url, -2)
add_bulleted_lists(slang_firearms_url, -4)

***REQUEST RAPGENIUS API***

***GET RAPGENIUS LYRICS***

use: get_song_lyrics(artist, song name)

In [10]:
#pass name of artist and song name as lowercase string and returns string of lyrics
def get_song_lyrics(artist, song_name):
    #set lyrics to lowercase
    artist_lower = artist.lower()
    song_lower = song_name.lower()
    
    if '|' in artist:
        artist_name = artist_lower.split("|")
        artist_lower = artist_name[1]
    if '$' in artist:
        artist_lower = re.sub(r'[$]', "-", artist_lower)
        print artist_lower
    if '.' in artist:
        artist_name = artist_lower.split(".")
        for x in artist_name:
            artist_lower = artist_lower + x
    if '!' in artist:
        artist_lower = re.sub(r'[!]', "", artist_lower)
    
    
    #remove special characters
    artist_lower = ''.join(e for e in artist_lower if e.isalnum() or e == " ")
    song_lower = ''.join(e for e in song_lower if e.isalnum() or e == " ")
    
    #capitalize artist name
    artist_capital = artist_lower.capitalize()
   
    #put dashes between names
    new_artist = "-".join(artist_capital.split())
    new_song_name = "-".join(song_lower.split())
    artist_and_song = "-".join([new_artist, new_song_name])
    
    #final url
    song_url = "https://genius.com/" + artist_and_song + "-lyrics"
    
    #gets lyrics and processes them
    lyrics = process_lyrics(getLyrics(song_url))
    
    #returns lyrics
    return lyrics
    

In [11]:
#**supports get_song_lyrics**
#cleans up lyrics once it gets sent back from rapgenius
#pass parameter as string lyrics
#returns as string of lyrics separated by spaces
def process_lyrics (lyrics):
    #removes stuff in brackets
    new_lyrics = re.sub("\[(.+?)\]", '', lyrics)
    
    #removes special characters and newline
    no_char = ''.join(e for e in new_lyrics if e.isalnum() or e == " " or e == "\n")
    no_char = " ".join(no_char.split("\n"))
    
    #returns string in lowercase
    return no_char.lower()

***ANALYZE SENTIMENT SCORE FOR LYRICS***

In [12]:
#input: artist as string and song name as string
#output: integer sentiment score for song
#necessary: needs get_song_lyrics and process_lyrics as auxillary methods
def analyze_senti(artist, song):
    lyrics_list = get_song_lyrics(artist, song).split(" ")
    
    value = 0
    numWords = 0 #words analyzed
    senti = 0
    for word in lyrics_list:
        if word in score_word_dict:
            value = value + int(score_word_dict[word])
            numWords = numWords + 1
        else:
            break
        senti = value / (1.0 * len(lyrics_list))
    return senti

In [13]:
print analyze_senti("xxxtentacion", "Fuck Love")
print analyze_senti("xxxtentacion", "jocelyn flores")
print analyze_senti("xxxtentacion", "run up on me")

-0.0662983425414
-0.0466926070039
-0.0882352941176


In [14]:
print analyze_senti("maxo kreaM", "hit mane")
print analyze_senti("03 greedo", "beat that thang down")

-0.0380952380952
-0.0157232704403


In [15]:
#input: artist as string for artist name, list of songs in album as list
#output: dict of album with senti scores
def process_album(artist, album):
    total = 0
    album_dict = {}
    for song in album:
        album_dict[song] = analyze_senti(artist, song)
    return album_dict

In [20]:
#input: artist as string and discography where album is key and album songs are list as value
#output: dict of artist discography where key is album and value is dictionary with songs as keys and senti scores as values 
def process_artist(artist, discography):
    total = 0
    artist_dict = {}
    for key, value in discography.iteritems():
        artist_dict[key] = process_album(artist, value)
    return artist_dict

In [21]:
#input: artist as string
#output: discography as dictionary where album are keys and songs are list of values
def get_disc(artist):
    url = "data/albums/%s.txt" % artist.decode("utf-8")
    with open(url, 'r') as f:
        x = eval(f.read())
    discography = {}
    for album in x: #x[0]
        discography[album["name"]] = [re.sub("\([^)]*\)", "", track['track_title']) for track in album['tracks']]
    return discography

In [22]:
def get_rappers():
    
    def validate(name):
        # checks if the name has a
        illegal_chars = "|?:^*\""

        for c in illegal_chars:
            if c == '|':
                if name.find(c) > 0:
                    name = name.split("|")[1]
            if c == "\"":
                if name.find(c) > 0:
                    name = name.replace(r'"','',2)
            else: 
                name = name.replace(c,'')
        return name
    
    with open("data/rappers.txt", "r") as f:
        rappers = f.read()
        
    return [validate(rapper).strip() for rapper in rappers.split("\n")]
rappers = get_rappers() 

In [38]:
"""x = get_rappers()

for artist in x:
    print artist
    try:
        discography = get_disc(artist)
        print discography
        senti_dict = process_artist(artist, discography)
        artist_url = "data/senti_data/%s.txt" % artist.decode("utf-8")
        file = open(artist_url, "w")
        file.write(str(senti_dict))
        file.close()
        print "done: " + artist
    except AttributeError, SyntaxError:
        continue"""

'x = get_rappers()\n\nfor artist in x:\n    print artist\n    try:\n        discography = get_disc(artist)\n        print discography\n        senti_dict = process_artist(artist, discography)\n        artist_url = "data/senti_data/%s.txt" % artist.decode("utf-8")\n        file = open(artist_url, "w")\n        file.write(str(senti_dict))\n        print "done: " + artist\n    except AttributeError, SyntaxError:\n        continue'

In [40]:
def save_lyrics(artist, discography):
    total = 0
    artist_dict = {}
    for key, value in discography.iteritems():
        artist_dict[key] = process_album(artist, value)
    return artist_dict

In [41]:
def process_album(artist, album):
    total = 0
    album_dict = {}
    for song in album:
        album_dict[song] = get_song_lyrics(artist, song)
    url = "data/lyrics/%s.txt" % artist
    file = open(url, 'w')
    file.write(str(album_dict))
    file.close()

In [44]:
em_disc = get_disc("drake")
x = save_lyrics("drake", em_disc)

AttributeError: 'NoneType' object has no attribute 'get_text'